# Importing Libraries 

In [ ]:
!pip install tensorflow-gpu
!pip install optuna

     |████████████████████████████████| 497.5 MB 27 kB/s 
     |████████████████████████████████| 462 kB 51.4 MB/s 
     |████████████████████████████████| 308 kB 5.2 MB/s 
     |████████████████████████████████| 210 kB 49.0 MB/s 
     |████████████████████████████████| 81 kB 8.2 MB/s 
     |████████████████████████████████| 75 kB 4.3 MB/s 
     |████████████████████████████████| 150 kB 47.4 MB/s 
     |████████████████████████████████| 49 kB 5.6 MB/s 
     |████████████████████████████████| 113 kB 49.9 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=4ddaded75367258be75dbe3bea9a208dca9031e19780a844ddb6a14d88db8cbf
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, Dropout, Conv2D, BatchNormalization, MaxPool2D, Flatten

# Dataset Preprocessing

## Importing Dataset

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Programs/Kaggle/Digit Recognizer/train.csv")
data_test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Programs/Kaggle/Digit Recognizer/test.csv")

## Splitting Into X and y variables

In [ ]:
X = data.iloc[:, 1:].values
y = data.iloc[:,0].values

In [ ]:
X_test = data_test.values
X_test.shape

(28000, 784)

## Encoding Categorical Features

In [ ]:
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding

y = to_categorical(y, num_classes = 10)

## Image Normalization

In [ ]:
X = X/255
X_test = X_test/255

## Reshaping

In [ ]:
X = X.reshape((42000,28,28,1))
X_test = X_test.reshape((28000,28,28,1))

## Splitting into training and validation

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_val,y_train, y_val = train_test_split(X, y, test_size = 0.1)

# Attempt 1: Creating Neural Network

## Building the CNN

In [ ]:
model_1 = tf.keras.models.Sequential()

In [ ]:
model_1.add(Conv2D(filters = 16, kernel_size = 3, activation = 'relu', input_shape = [28,28,1]))
model_1.add(BatchNormalization())
model_1.add(Conv2D(filters = 16, kernel_size = 3, activation = 'relu'))
model_1.add(BatchNormalization())
model_1.add(MaxPool2D(pool_size = 2, strides = 2, padding = 'valid'))

In [ ]:
model_1.add(Conv2D(filters = 16, kernel_size = 3, activation = 'relu'))
model_1.add(Conv2D(filters = 32, kernel_size = 3, activation = 'relu'))
model_1.add(MaxPool2D(pool_size = 2, strides = 2, padding = 'valid'))

In [ ]:
model_1.add(Flatten())
model_1.add(BatchNormalization())

In [ ]:
model_1.add(Dense(units = 32, activation = 'relu'))
model_1.add(BatchNormalization())
model_1.add(Dropout(rate = 0.1))
model_1.add(Dense(units = 64, activation = 'relu'))

In [ ]:
model_1.add(Dense(units = 10, activation = 'softmax'))


In [ ]:
model_1.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer="adam", metrics=["accuracy"])

## Training the CNN

In [ ]:
model_1.fit(X_train, y_train, epochs = 3)

Epoch 1/3
1182/1182 [==============================] - 55s 46ms/step - loss: 0.2207 - accuracy: 0.9378
Epoch 2/3
1182/1182 [==============================] - 54s 46ms/step - loss: 0.0754 - accuracy: 0.9775
Epoch 3/3
1182/1182 [==============================] - 54s 46ms/step - loss: 0.0597 - accuracy: 0.9809


In [ ]:
accuracy = model_1.evaluate(X_val, y_val)
accuracies = [accuracy[1]]

132/132 [==============================] - 2s 12ms/step - loss: 0.0621 - accuracy: 0.9810


## Testing the CNN

# Attempt 2: Optuna Hyperparameter Tuning with a DCNN

## Network One

Layers:

1. Conv_2d
2. Conv_2d
3. Max_pooling
4. Conv_2d
5. Conv_2d
6. Max_Pooling
7. Flatten
8. Dense_1
9. Dropout
10. Dense
11. Output





In [ ]:
import optuna
def objective_NN(trial):
  conv_2d_1 = Conv2D(filters = trial.suggest_int("conv_2d_1_filters" ,8, 128),
                     kernel_size = 3,
                     activation ='relu',
                     input_shape = (28, 28, 1))
  conv_2d_2 = Conv2D(filters = trial.suggest_int("conv_2d_2_filters" ,8, 128),
                     kernel_size =3,
                     activation = 'relu')
  conv_2d_3 = Conv2D(filters = trial.suggest_int("conv_2d_3_filters" ,8, 128),
                     kernel_size = 3,
                     activation = 'relu')
  conv_2d_4 = Conv2D(filters = trial.suggest_int("conv_2d_4_filters" ,8, 256),
                     kernel_size = 3,
                     activation = 'relu')
  
  max_pooling_1 = MaxPool2D(pool_size = 2, strides = 2, padding = 'valid')
  max_pooling_2 = MaxPool2D(pool_size = 2, strides = 2, padding = 'valid')

  flatten = Flatten()

  dense_1 = Dense(units = trial.suggest_int("dense_1_units", 16, 256), activation = trial.suggest_categorical('dense_1_activation', ['relu', 'linear', 'sigmoid']) )
  dense_2 = Dense(units = trial.suggest_int("dense_2_units", 16, 256), activation = trial.suggest_categorical('dense_2_activation', ['relu', 'linear', 'sigmoid']) )

  dropout = Dropout(rate = trial.suggest_float('dropout_rate', 0.05, 1))

  output = Dense(units = 10, activation = trial.suggest_categorical('output_activation', ['softmax', 'sigmoid']))



  model_layers = (conv_2d_1,conv_2d_2, max_pooling_1, conv_2d_3, conv_2d_4, max_pooling_2, flatten, dense_1, dropout, dense_2, output)
  model = tf.keras.models.Sequential(model_layers)
  model.compile(loss = 'categorical_crossentropy', optimizer="adam", metrics=["accuracy"])

  model.fit(X_train, y_train, epochs = trial.suggest_int('epochs', 1, 10))

  accuracy = model.evaluate(X_val, y_val)

  return accuracy[1]

study = optuna.create_study(direction='maximize')
study.optimize(objective_NN, n_trials = 100)

[I 2021-05-21 15:35:53,884] A new study created in memory with name: no-name-31ed89ac-0a7b-4f4d-b75b-d1241d22bdca


Epoch 1/6
1182/1182 [==============================] - 145s 122ms/step - loss: 0.2078 - accuracy: 0.9379
Epoch 2/6
1182/1182 [==============================] - 143s 121ms/step - loss: 0.0576 - accuracy: 0.9831
Epoch 3/6
1182/1182 [==============================] - 143s 121ms/step - loss: 0.0391 - accuracy: 0.9879
Epoch 4/6
1182/1182 [==============================] - 142s 120ms/step - loss: 0.0320 - accuracy: 0.9899
Epoch 5/6
1182/1182 [==============================] - 142s 120ms/step - loss: 0.0272 - accuracy: 0.9915
Epoch 6/6
132/132 [==============================] - 5s 33ms/step - loss: 0.0621 - accuracy: 0.9807


[I 2021-05-21 15:50:16,852] Trial 0 finished with value: 0.9807142615318298 and parameters: {'conv_2d_1_filters': 22, 'conv_2d_2_filters': 76, 'conv_2d_3_filters': 73, 'conv_2d_4_filters': 93, 'dense_1_units': 67, 'dense_1_activation': 'sigmoid', 'dense_2_units': 178, 'dense_2_activation': 'relu', 'dropout_rate': 0.16682934581614878, 'output_activation': 'softmax', 'epochs': 6}. Best is trial 0 with value: 0.9807142615318298.


Epoch 1/7
1182/1182 [==============================] - 191s 161ms/step - loss: 1.5415 - accuracy: 0.4210
Epoch 2/7
1151/1182 [============================>.] - ETA: 4s - loss: 1.1466 - accuracy: 0.5295

KeyboardInterrupt: ignored

In [ ]:
#  Trial 21 finished with value: 0.9933333396911621 and parameters: {'conv_2d_1_filters': 67, 'conv_2d_2_filters': 104, 'conv_2d_3_filters': 74, 'conv_2d_4_filters': 82, 'dense_1_units': 136, 'dense_1_activation': 'relu', 'dense_2_units': 252, 'dense_2_activation': 'sigmoid', 'dropout_rate': 0.7265591982474168, 'output_activation': 'sigmoid', 'epochs': 10}'sigmoid', 'dense_2_units': 96, 'dense_2_activation': 'sigmoid', 'dropout_rate': 0.5701146378876534, 'output_activation': 'softmax', 'epochs': 10}. Best is trial 37 with value: 0.9930952191352844.
print("Number of finished trials: {}".format(len(study.trials)))
print("Best trial:")
trial = study.best_trial
print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
  print("    {}: {}".format(key, value))

In [ ]:
optuna.visualization.plot_param_importances(study)

In [ ]:
def build_model(conv_2d_1_filters, conv_2d_2_filters, conv_2d_3_filters, conv_2d_4_filters, dense_1_units, dense_1_activation, dense_2_units, dense_2_activation, dropout_rate, output_activation):
  conv_2d_1 = Conv2D(filters = conv_2d_1_filters,
                     kernel_size = 3,
                     activation ='relu',
                     input_shape = (28, 28, 1))
  conv_2d_2 = Conv2D(filters = conv_2d_2_filters,
                     kernel_size = 3,
                     activation = 'relu')
  conv_2d_3 = Conv2D(filters = conv_2d_3_filters,
                     kernel_size = 3,
                     activation = 'relu')
  conv_2d_4 = Conv2D(filters = conv_2d_4_filters,
                     kernel_size = 3,
                     activation = 'relu')
  
  max_pooling_1 = MaxPool2D(pool_size = 2, strides = 2, padding = 'valid')
  max_pooling_2 = MaxPool2D(pool_size = 2, strides = 2, padding = 'valid')

  flatten = Flatten()

  dense_1 = Dense(units = dense_1_units, activation = dense_1_activation )
  dense_2 = Dense(units = dense_2_units, activation = dense_2_activation)

  dropout = Dropout(rate = dropout_rate)

  output = Dense(units = 10, activation = output_activation)



  model_layers = (conv_2d_1,conv_2d_2, max_pooling_1, conv_2d_3, conv_2d_4, max_pooling_2, flatten, dense_1, dropout, dense_2, output)
  return tf.keras.models.Sequential(model_layers)


In [ ]:
params = study.best_trial.params
conv_2d_1_filters= params['conv_2d_1_filters'] 
conv_2d_2_filters= params['conv_2d_2_filters']
conv_2d_3_filters= params['conv_2d_3_filters']
conv_2d_4_filters= params['conv_2d_4_filters']
dense_1_units= params['dense_1_units']
dense_1_activation= params['dense_1_activation']
dense_2_units= params['dense_2_units']
dense_2_activation= params['dense_2_activation']
dropout_rate= params['dropout_rate']
output_activation = params['output_activation']
model_2 = build_model(conv_2d_1_filters, conv_2d_2_filters, conv_2d_3_filters, conv_2d_4_filters, dense_1_units, dense_1_activation, dense_2_units, dense_2_activation, dropout_rate, output_activation)
model_2.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer="adam", metrics=["accuracy"])


In [ ]:
model_2.fit(X_train, y_train, epochs = params['epochs'])

In [ ]:
model_2.evaluate(X_val, y_val)

# Creating Testing Results

In [ ]:
predictions = model_2.predict_classes(X_test)

In [ ]:
predictions

In [ ]:
submission = []
num = 1
for x in predictions:
  submission.append([num, x])
  num += 1

In [ ]:
submission = pd.DataFrame(submission)
submission.columns = ["ImageId", "Label"]

In [ ]:
submission.to_csv("output.csv", index = False)

In [ ]:
submission